In [1]:
#Import of Python available libraries
import sculib

mpi = sculib.scu()
#mpi.ip = '10.96.64.10'
mpi.ip = '10.96.254.85'

In [2]:
def mpi_t6103az(scu):

    #Feed Indexer tests [316-000000-043] 
    print('Test 6.10.3 az only')

    #Revisions:
    #0.1 initial rewrite

    #Common setup
    config_name = 'HN_INDEX_TEST'
  
    loop1 = 10

    #elevations = [15, 45, 85] lower limit not accessible
    az_start = -90
    az_speed = 3
    el_start = 20 #15
    el_speed = 0.1
    elevations = [20]
    
    wait5 = 5
    wait10 = 10
    wait100 = 100
    sampling = 200
    scu.debug = False

    #test only remove these
    '''
    wait5 = 1
    wait10 = 1
    wait100 = 1
    sampling = 5000
    scu.ip = '10.96.254.85'
    scu.debug = True
    '''
    #Feed Indexer: Response & Repeatability (6.8.8)
    '''
        a. Feed Indexer: Response & Repeatability (6.8.8)
        1) Manually move the Dish to 15° Elevation, any Azimuth angle (suitable for the current site conditions) and FI at SPF Band5c and de-activate all axes.
        2) Manually start the LDS data recording
        3) Start the ACU data recording on the SCU using logging configuration TBD 
        4) Command the FI to the SPF B1 position,
            ◦  wait for 40 seconds to complete the move
        5) Command the FI to the SPF B2 position, 
            ◦ wait for 40 seconds to complete the move
        6) Command the FI to the SPF Band 5c position,
            ◦ wait for 100 seconds to complete the move
        8) Command the FI to SPF B1 position, 
            ◦ wait 50 seconds to complete the move 
        9) Command the FI to SPF B5c position, 
            ◦ wait 100 seconds to complete the move and to stabilise
        10) Command the FI to SPF B2 position, 
            ◦ wait 50 seconds to complete the move 
        11) Command the FI to SPF B5c position, 
            ◦ wait 100 seconds to complete the move and to stabilise
        12) Repeat steps 7) to 10) for a total of 10 cycles
        13) Command the FI to SPF B1 position, 
            ◦ wait 40 seconds to complete the move 
        14) Command the FI to SPF B2 position, 
            ◦ wait 100 seconds to complete the move 
        15) Command the FI to SPF B1 position, 
            ◦ wait 100 seconds to complete the move 
        16) Repeat steps 13) to 14)  for a total of 5 cycles 
        17) Command the FI to SPF B5c position, 
            ◦ wait 40 seconds to complete the move
        18) Stop the ACU data recording on the SCU
        19) Manually stop the LDS data recordings
        20) Extract the data on the SCU @ 5Hz sampling
        21) Rename the file to TBD (eg. 6p8p8_i.csv)
        22) Repeat the sequence 1) to 19) at 45° Elevation.
        23) Repeat the sequence 1) to 19) at 85° Elevation.
    (Total duration ~ 1h per elevation angle => 3h)
    '''
    #Step 1
    scu.activate_dmc()
    scu.wait_duration(wait10*3)

    #Move to starting az
    scu.abs_azimuth(az_start, az_speed)
    scu.wait_duration(wait10)

    #Move to starting el
    scu.abs_elevation(el_start, el_speed)
    scu.wait_duration(wait10*3)

    #Move to starting band
    band = 'Band 5c'
    scu.move_to_band(band)
    scu.wait_duration(wait10*4)

    for el_angle in elevations:
        #Step 2
        print('Start LDS recording here:')
        #input()

        print('Select elevation angle: {:.3f}  \n'.format(el_angle))
        scu.abs_elevation(el_angle, el_speed)
        scu.wait_duration(wait10*4)

        #Step 3
        if scu.logger_state() == 'RECORDING':
            print('WARNING, already recording - attempting to stop and start a fresh logger')
            scu.stop_logger()  
            scu.wait_duration(wait5)
        if scu.logger_state() == 'STOPPED':
            print('Starting logger with config: {}'.format(config_name))
            scu.start_logger(config_name)
            scu.wait_duration(wait5)

        #Step 4
        band = 'Band 1'
        scu.move_to_band(band)
        scu.wait_duration(wait10*5)

        #Step 5
        band = 'Band 2'
        scu.move_to_band(band)
        scu.wait_duration(wait10*5)

        #Step 6
        band = 'Band 5c'
        scu.move_to_band(band)
        scu.wait_duration(wait100)

        #Step 7 deleted but start of loop


        #Initial manual test in azimuth to verify LDS setup correct
        i=0
        while i < loop1:
            i+=1
            print('Loop {}:'.format(i))
        #Move to starting az
            scu.abs_azimuth(az_start, az_speed)
            scu.wait_duration(wait10*6)

            scu.abs_azimuth(az_start-10, az_speed)
            scu.wait_duration(wait5)

            scu.abs_azimuth(az_start, az_speed)
            scu.wait_duration(wait10*3)

            scu.abs_azimuth(az_start+10, az_speed)
            scu.wait_duration(wait5)

   
        #Step 18
        scu.stop_logger()
        scu.wait_duration(wait5)
        print(scu.logger_state())

        #Step 19
        print('Stop LDS recording now :')    
        #input()
        #Step 20
        print('Extract the data on the SCU @ 5Hz sampling')
        scu.save_session('6p3p10_i_az-only_elev_' + str(el_angle), interval_ms = sampling)
        #Step 21
        print('Rename the file to TBD (eg. 6p3p10_i.csv)')
        #Step 22
        #print('Repeat the sequence 1) to 19) at 45° Elevation.')
        #Step 23
        #print('Repeat the sequence 1) to 19) at 85° Elevation.')
    print('Dropping elevation')
    scu.abs_elevation(el_start, el_speed)
    scu.wait_duration(wait10*3)
    print('Done')
    scu.deactivate_dmc()
    scu.wait_duration(wait10*3)


In [3]:
mpi_t6103az(mpi)


Test 6.10.3 az only
activate dmc
wait for 3.0s done *
abs az: -90.000 vel: 3.000
wait for 1.0s done *
abs el: 20.000 vel: 0.100
wait for 3.0s done *
move to band: Band 5c
wait for 4.0s done *
Start LDS recording here:
Select elevation angle: 20.000  

abs el: 20.000 vel: 0.100
wait for 4.0s done *
logger state 
logger state 
Starting logger with config: HN_INDEX_TEST
start logger:  HN_INDEX_TEST
wait for 1.0s done *
move to band: Band 1
wait for 5.0s done *
move to band: Band 2
wait for 5.0s done *
move to band: Band 5c
wait for 1.0s done *
Loop 1:
abs az: -90.000 vel: 3.000
wait for 6.0s done *
abs az: -100.000 vel: 3.000
wait for 1.0s done *
abs az: -90.000 vel: 3.000
wait for 3.0s done *
abs az: -80.000 vel: 3.000
wait for 1.0s done *
Loop 2:
abs az: -90.000 vel: 3.000
wait for 6.0s done *
abs az: -100.000 vel: 3.000
wait for 1.0s done *
abs az: -90.000 vel: 3.000
wait for 3.0s done *
abs az: -80.000 vel: 3.000
wait for 1.0s done *
Loop 3:
abs az: -90.000 vel: 3.000
wait for 6.0s do